# Desafio 2 - Ciência e Governança de Dados
## Universidade Federal de Lavras  

### Autor: **Paulo Henrique dos Anjos Silveira** - 202310533  

---

# Modelos Preditivos de Machine Learning

## Objetivo
O presente trabalho tem como objetivo desenvolver modelos robustos e interpretáveis para prever impactos socioeconômicos nos municípios da Região Sudeste do Brasil baseados em agentes e fenômenos identificados na análise exploratória. Conforme proposto no Desafio II – Ciência e Governança de Dados da Zetta Lab/UFLA.

### Seleção de Modelos
Para esta tarefa, selecionamos 3 abordagens complementares:

1. **Random Forest**
2. **Gradient Boosting (XGBoost)**
3. **Ridge Regression**

In [1]:
# Importações para ML
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

## 1. Preparação dos Dados para Modelagem
### Importando biblitecas necessárias

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import randint, uniform

### Recupera o DataFrame salvo em CSV


In [ ]:
csv_path = r"c:\Users\paulo\Downloads\Desafio-1-Ciencia-e-Governanca-de-Dados\dataFrame.csv"
if os.path.exists(csv_path):
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
        print(f"Carregado {len(df):,} linhas em `df` a partir de: {csv_path}")
    except Exception as e:
        print("Erro ao ler CSV:", e)
else:
    print(f"Arquivo não encontrado: {csv_path}. Verifique o caminho ou gere o CSV no outro notebook.")

Carregado 1,661 linhas em `df` a partir de: c:\Users\paulo\Downloads\Desafio-1-Ciencia-e-Governanca-de-Dados\dataFrame.csv


### Mudanças no dataset e justificativa

- Alterações feitas no dataset:
  - Adicionadas colunas: `pib_per_capita`, `log_pib_pc`.
  - `populacao` foi mantida no DataFrame, mas removida da lista de preditores.
  - Substituímos `pib` por `log_pib_pc` na lista `variaveis_preditoras`.

- Justificativa:
  - O PIB bruto apresenta escala muito maior e alta correlação com `populacao`.
  - Usar PIB per capita (e aplicar log1p) melhora interpretabilidade e reduz
    multicolinearidade, tornando os coeficientes mais estáveis e coerentes.

In [ ]:
try:
    df['pib_per_capita'] = df['pib'] / df['populacao']
    df['log_pib_pc'] = np.log1p(df['pib_per_capita'])
    print("Transformações aplicadas: 'pib_per_capita', 'log_pib_pc'")
except Exception as e:
    print("Erro ao criar transformações de PIB (verifique colunas 'pib' e 'populacao'):", e)

Transformações aplicadas: 'pib_per_capita', 'log_pib_pc'


### Função para preparar dados para modelagem

#### Por que utlizei o IDHM como impacto socioeconômico principal?

> O IDHM foi adotado como principal variável de impacto socioeconômico por ser uma medida multidimensional que integra renda, educação e longevidade, permitindo comparações consistentes entre municípios e oferecendo alta relevância para a formulação de políticas públicas. Além disso, por ser um indicador agregado, reduz o ruído de variáveis isoladas e facilita a interpretação dos resultados. Como limitação, o IDHM pode refletir efeitos com defasagem temporal, de modo que as relações observadas indicam associações contemporâneas, e não causalidade imediata.

<div class="alert alert-info">
  <strong>Nota sobre a variável “desemprego”:</strong><br>
  A coluna <em>desemprego</em> apresenta valores constantes (mesma média para todos os municípios), não oferecendo variabilidade informativa para a modelagem.
  Para evitar memorização perfeita do modelo (R² = 1.0), essa variável foi removida do conjunto de preditores.
  A coluna original foi mantida no <em>DataFrame</em> apenas para fins de referência futura.
</div>


In [ ]:
def preparar_dados_ml(df, variavel_alvo, variaveis_preditoras):
    """
    Prepara dados para treinamento de modelos ML
    """
    X = df[variaveis_preditoras]
    y = df[variavel_alvo]
    
    # Verifica variância das features
    variancias = X.var()
    print(f"\n  Análise das Features:")
    print(f"   Total de amostras: {len(X)}")
    print(f"   Features com variância > 0: {(variancias > 0).sum()}/{len(variancias)}")
    print(f"   Features: {list(X.columns)}")
    
    return X, y, df

# Usando IDHM como variável alvo
print("\nPredição de IDHM (Desenvolvimento Humano Municipal)")

variaveis_preditoras = ['log_pib_pc', 'internet', 'homicidios', 'analfabetismo']
variavel_alvo = 'idhm'

X, y, dados = preparar_dados_ml(df, variavel_alvo, variaveis_preditoras)

print(f"\n Dados prontos para modelagem")
print(f"   Variável alvo: {variavel_alvo}")
print(f"   IDHM - Min: {y.min():.3f}, Max: {y.max():.3f}, Média: {y.mean():.3f}")


Predição de IDHM (Desenvolvimento Humano Municipal)

  Análise das Features:
   Total de amostras: 1661
   Features com variância > 0: 4/4
   Features: ['log_pib_pc', 'internet', 'homicidios', 'analfabetismo']

 Dados prontos para modelagem
   Variável alvo: idhm
   IDHM - Min: 0.762, Max: 0.806, Média: 0.786


### Hiperparâmetros

In [ ]:
param_dist_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None]
}

param_dist_xgb = {
    'n_estimators': randint(100,300),
    'max_depth': randint(3,10),
    'learning_rate': uniform(0.01,0.3),
    'subsample': uniform(0.6,0.4),
    'colsample_bytree': uniform(0.6,0.4)
}

param_dist_ridge = {'alpha': uniform(0.001, 10)}

## 2. Divisão em Treino/Teste e Normalização

In [ ]:
# Divisão treino/teste com estratificação por estado
X_train, X_test, y_train, y_test, estados_train, estados_test = train_test_split(
    X, y, dados['sigla'], test_size=0.2, random_state=42
)

# Normalização das features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(" DIVISÃO DOS DADOS")

print(f"Treino: {len(X_train)} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"Teste:  {len(X_test)} amostras ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nFeatures normalizadas")
print(f"Média após normalização: {X_train_scaled.mean(axis=0).mean():.6f}")
print(f"Desvio padrão após normalização: {X_train_scaled.std(axis=0).mean():.6f}")

 DIVISÃO DOS DADOS
Treino: 1328 amostras (80.0%)
Teste:  333 amostras (20.0%)

Features normalizadas
Média após normalização: 0.000000
Desvio padrão após normalização: 1.000000


## 3. Modelo 1: Random Forest Regressor (Não-linear)

### Por que usar Random Forest?

> O Random Forest foi escolhido por capturar relações não lineares complexas entre os indicadores e o IDHM, ser robusto a outliers, dispensar normalização dos dados e reduzir overfitting por meio do uso de ensembles. Além disso, oferece medidas interpretáveis de importância das variáveis, facilitando a análise e a comunicação dos resultados.

### Fórmula Geral
$$\hat{y}_{RF} = \frac{1}{B} \sum_{b=1}^{B} T_b(x)$$

Onde:
- $B$ = número de árvores (200)
- $T_b(x)$ = predição da árvore $b$
- $\hat{y}_{RF}$ = média das predições

### Feature Importance (Mean Decrease Impurity)
$$\text{Importance}_j = \frac{1}{B} \sum_{b=1}^{B} \sum_{t \in T_b} \mathbb{1}[\text{split}_t = j] \times \Delta \text{Impurity}_t$$

Onde:
- $j$ = feature de interesse
- $\Delta \text{Impurity}_t$ = redução de impureza no split $t$

In [ ]:
# RandomizedSearchCV para Random Forest
rf_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist_rf,
    n_iter=20,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
rf_search.fit(X_train, y_train)
rf_model = rf_search.best_estimator_
print("Melhores parâmetros (RF):", rf_search.best_params_)

# Predições
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Métricas
r2_train_rf = r2_score(y_train, y_train_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)
rmse_test_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))
mae_test_rf = mean_absolute_error(y_test, y_test_pred_rf)

# Validação Cruzada
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='r2')

print(f"\n Desempenho:")
print(f"   R² Treino: {r2_train_rf:.4f}")
print(f"   R² Teste:  {r2_test_rf:.4f}")
print(f"   RMSE:      {rmse_test_rf:.4f}")
print(f"   MAE:       {mae_test_rf:.4f}")
print(f"\n Validação Cruzada (5-fold):")
print(f"   R² Médio: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

# Feature Importance
feature_importance_rf = pd.DataFrame({
    'Feature': variaveis_preditoras,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n Feature Importance (Random Forest):")
for idx, row in feature_importance_rf.iterrows():
    pct = row['Importance'] * 100
    print(f"   {row['Feature']:20s}: {pct:6.2f}%")

# Visualização de Feature Importance
fig_rf_imp = px.bar(
    feature_importance_rf,
    x='Importance',
    y='Feature',
    orientation='h',
    title='Random Forest: Importância das Features',
    labels={'Importance': 'Importância Relativa', 'Feature': 'Variável'},
    color='Importance',
    color_continuous_scale='Viridis'
)
fig_rf_imp.update_layout(showlegend=False, height=400)
fig_rf_imp.show()


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Melhores parâmetros (RF): {'max_depth': 9, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 430}

 Desempenho:
   R² Treino: 0.6049
   R² Teste:  0.3187
   RMSE:      0.0132
   MAE:       0.0100

 Validação Cruzada (5-fold):
   R² Médio: 0.3126 (+/- 0.0458)

 Feature Importance (Random Forest):
   analfabetismo       :  39.03%
   internet            :  26.35%
   log_pib_pc          :  18.84%
   homicidios          :  15.77%


In [ ]:
# Análise de resíduos do Random Forest
residuos_rf = y_test - y_test_pred_rf

fig_rf_resid = go.Figure()
fig_rf_resid.add_trace(go.Scatter(
    x=y_test_pred_rf,
    y=residuos_rf,
    mode='markers',
    marker=dict(size=8, color='rgba(31, 119, 180, 0.6)'),
    name='Resíduos'
))
fig_rf_resid.add_hline(y=0, line_dash='dash', line_color='red')
fig_rf_resid.update_layout(
    title='Random Forest: Análise de Resíduos',
    xaxis_title='Valores Preditos',
    yaxis_title='Resíduos',
    height=400
)
fig_rf_resid.show()

### Resultados e Análise

> O Random Forest demonstrou boa capacidade de generalização, sem sinais fortes de overfitting, e produziu interpretações coerentes do ponto de vista socioeconômico. Destaca-se positivamente por capturar relações não lineares e indicar variáveis relevantes para políticas públicas. Como limitação, o modelo não explica completamente o fenômeno estudado, sugerindo que o desenvolvimento humano depende de fatores adicionais não contemplados nos dados.

## 4. Modelo 2: XGBoost (Gradient Boosting)

### Por que usar XGBoost?

> O XGBoost foi utilizado como complemento ao Random Forest por sua capacidade de aprender padrões mais sutis por meio de treinamento sequencial e regularização explícita. Destaca-se pela boa generalização em conjuntos de dados menores, alto desempenho prático e maior controle sobre a complexidade do modelo.

### Objetivo de Otimização
$$\text{Loss} = \sum_{i=1}^{n} l(y_i, \hat{y}_i) + \sum_{k=1}^{K} \Omega(f_k)$$

Onde:
- $l()$ = função de perda (squared error para regressão)
- $\Omega(f)$ = termo de regularização (penaliza complexidade)

### Predição Iterativa
$$\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + \eta \cdot f_t(x_i)$$

Onde:
- $t$ = iteração (1 a 150)
- $\eta$ = learning rate (0.1)
- $f_t()$ = árvore treinada na iteração $t$

### Regularização
$$\Omega(f) = \gamma T + \frac{1}{2}\lambda \|w\|^2$$

Onde:
- $T$ = número de folhas
- $\gamma$ = penalidade por complexidade
- $\lambda$ = penalidade L2 dos pesos

In [ ]:
xgb_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42, verbosity=0, tree_method='hist'),
    param_distributions=param_dist_xgb,
    n_iter=30,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

xgb_search.fit(X_train, y_train)
xgb_model = xgb_search.best_estimator_
print('Melhores parâmetros (XGB):', xgb_search.best_params_)

# Predições
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

# Métricas
r2_train_xgb = r2_score(y_train, y_train_pred_xgb)
r2_test_xgb = r2_score(y_test, y_test_pred_xgb)
rmse_test_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
mae_test_xgb = mean_absolute_error(y_test, y_test_pred_xgb)

# Validação Cruzada
cv_scores_xgb = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='r2')

print(f"\n Desempenho:")
print(f"   R² Treino: {r2_train_xgb:.4f}")
print(f"   R² Teste:  {r2_test_xgb:.4f}")
print(f"   RMSE:      {rmse_test_xgb:.4f}")
print(f"   MAE:       {mae_test_xgb:.4f}")
print(f"\n Validação Cruzada (5-fold):")
print(f"   R² Médio: {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std():.4f})")

# Feature Importance
feature_importance_xgb = pd.DataFrame({
    'Feature': variaveis_preditoras,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n Feature Importance (XGBoost):")
for idx, row in feature_importance_xgb.iterrows():
    pct = row['Importance'] * 100
    print(f"   {row['Feature']:20s}: {pct:6.2f}%")

# Visualização
fig_xgb_imp = px.bar(
    feature_importance_xgb,
    x='Importance',
    y='Feature',
    orientation='h',
    title='XGBoost: Importância das Features (Ganho Médio)',
    labels={'Importance': 'Importância Relativa', 'Feature': 'Variável'},
    color='Importance',
    color_continuous_scale='Plasma'
)
fig_xgb_imp.update_layout(showlegend=False, height=400)
fig_xgb_imp.show()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Melhores parâmetros (XGB): {'colsample_bytree': np.float64(0.7599443886861021), 'learning_rate': np.float64(0.023999698964084628), 'max_depth': 6, 'n_estimators': 114, 'subsample': np.float64(0.7824279936868144)}

 Desempenho:
   R² Treino: 0.6120
   R² Teste:  0.3118
   RMSE:      0.0132
   MAE:       0.0105

 Validação Cruzada (5-fold):
   R² Médio: 0.2907 (+/- 0.0425)

 Feature Importance (XGBoost):
   analfabetismo       :  37.66%
   homicidios          :  26.28%
   internet            :  21.11%
   log_pib_pc          :  14.95%


In [ ]:
# Análise de resíduos do XGBoost
residuos_xgb = y_test - y_test_pred_xgb

fig_xgb_resid = go.Figure()
fig_xgb_resid.add_trace(go.Scatter(
    x=y_test_pred_xgb,
    y=residuos_xgb,
    mode='markers',
    marker=dict(size=8, color='rgba(255, 127, 14, 0.6)'),
    name='Resíduos'
))
fig_xgb_resid.add_hline(y=0, line_dash='dash', line_color='red')
fig_xgb_resid.update_layout(
    title='XGBoost: Análise de Resíduos',
    xaxis_title='Valores Preditos',
    yaxis_title='Resíduos',
    height=400
)
fig_xgb_resid.show()

### Resultados e Análise

> O XGBoost apresentou boa capacidade de ajuste no treino, mas com generalização mais limitada no conjunto de teste. O desempenho consistente na validação cruzada indica estabilidade moderada, embora ainda haja perda de explicação fora da amostra. O modelo destacou variáveis sociais como mais relevantes, sugerindo sensibilidade a fatores estruturais. No geral, é um modelo expressivo, porém mais suscetível a overfitting que abordagens mais simples.

## 5. Modelo 3: Ridge Regression

### Por que usar Ridge Regression?

> O Ridge Regression traz uma abordagem simples e altamente interpretável, facilitando a comunicação dos resultados. É computacionalmente eficiente e estatisticamente bem fundamentado, oferecendo estimativas estáveis mesmo com variáveis correlacionadas. Atua como um bom modelo de referência para avaliar se métodos mais complexos realmente agregam valor. Além disso, ajuda a verificar se as relações entre variáveis podem ser bem descritas por um modelo linear.

### Objetivo de Otimização (L2 Regularization)
$$J(\beta) = \sum_{i=1}^{n} (y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij})^2 + \alpha \sum_{j=1}^{p} \beta_j^2$$

Ou em forma matricial:
$$J(\beta) = (y - X\beta)^T(y - X\beta) + \alpha \beta^T \beta$$

### Solução Analítica
$$\hat{\beta}_{Ridge} = (X^T X + \alpha I)^{-1} X^T y$$

Onde:
- $\alpha$ = parâmetro de regularização (0.5 neste caso)
- $I$ = matriz identidade

### Propriedades Estatísticas
$$\mathbb{E}[\hat{\beta}_{Ridge}] \neq \beta \quad \text{(enviesado)}$$
$$\text{Var}[\hat{\beta}_{Ridge}] < \text{Var}[\hat{\beta}_{OLS}] \quad \text{(variância menor)}$$

O trade-off: aceita pequeno viés para ganhar variância muito menor (bias-variance trade-off).

### Interpretação dos Coeficientes
Para dados **normalizados** (média 0, desvio padrão 1):

Aumento de 1 desvio padrão em $x_j$ → aumento de $\hat{\beta}_j$ em $y$

$$\Delta y = \hat{\beta}_j \times \Delta x_j$$

In [ ]:
# Treinamento do Ridge
ridge_search = RandomizedSearchCV(
    estimator=Ridge(), 
    param_distributions=param_dist_ridge, 
    n_iter=30, 
    cv=5, 
    scoring='r2', 
    random_state=42, 
    n_jobs=-1
)
ridge_search.fit(X_train_scaled, y_train)
ridge_model = ridge_search.best_estimator_
print('Best Ridge params:', ridge_search.best_params_)

# Predições
y_train_pred_ridge = ridge_model.predict(X_train_scaled)
y_test_pred_ridge = ridge_model.predict(X_test_scaled)

# Métricas
r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))
mae_test_ridge = mean_absolute_error(y_test, y_test_pred_ridge)

# Validação Cruzada
cv_scores_ridge = cross_val_score(ridge_model, X_train_scaled, y_train, cv=5, scoring='r2')

print(f"\n Desempenho:")
print(f"   R² Treino: {r2_train_ridge:.4f}")
print(f"   R² Teste:  {r2_test_ridge:.4f}")
print(f"   RMSE:      {rmse_test_ridge:.4f}")
print(f"   MAE:       {mae_test_ridge:.4f}")
print(f"\n Validação Cruzada (5-fold):")
print(f"   R² Médio: {cv_scores_ridge.mean():.4f} (+/- {cv_scores_ridge.std():.4f})")

# Coeficientes
coeficientes_ridge = pd.DataFrame({
    'Feature': variaveis_preditoras,
    'Coeficiente': ridge_model.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)

print(f"\n Coeficientes (dados normalizados):")
print(f"   Intercepto: {ridge_model.intercept_:.6f}")
print(f"\n   Variável                Coeficiente")
print(f"   " + "-" * 40)
for idx, row in coeficientes_ridge.iterrows():
    coef = row['Coeficiente']
    print(f"   {row['Feature']:20s}  {coef:+10.6f}")

# Visualização dos coeficientes
fig_ridge_coef = px.bar(
    coeficientes_ridge,
    x='Coeficiente',
    y='Feature',
    orientation='h',
    title='Ridge Regression: Coeficientes Padronizados',
    labels={'Coeficiente': 'Valor do Coeficiente', 'Feature': 'Variável'},
    color='Coeficiente',
    color_continuous_scale='RdBu',
    color_continuous_midpoint=0
)
fig_ridge_coef.update_layout(showlegend=False, height=400)
fig_ridge_coef.show()

Best Ridge params: {'alpha': np.float64(9.700098521619942)}

 Desempenho:
   R² Treino: 0.2112
   R² Teste:  0.1792
   RMSE:      0.0145
   MAE:       0.0132

 Validação Cruzada (5-fold):
   R² Médio: 0.1751 (+/- 0.0493)

 Coeficientes (dados normalizados):
   Intercepto: 0.785769

   Variável                Coeficiente
   ----------------------------------------
   analfabetismo          -0.005453
   internet               +0.002467
   homicidios             -0.002176
   log_pib_pc             +0.000343


In [ ]:
# Análise de resíduos do Ridge
residuos_ridge = y_test - y_test_pred_ridge

fig_ridge_resid = go.Figure()
fig_ridge_resid.add_trace(go.Scatter(
    x=y_test_pred_ridge,
    y=residuos_ridge,
    mode='markers',
    marker=dict(size=8, color='rgba(44, 160, 44, 0.6)'),
    name='Resíduos'
))
fig_ridge_resid.add_hline(y=0, line_dash='dash', line_color='red')
fig_ridge_resid.update_layout(
    title='Ridge Regression: Análise de Resíduos',
    xaxis_title='Valores Preditos',
    yaxis_title='Resíduos',
    height=400
)
fig_ridge_resid.show()

### Resultados e Análise

> O Ridge apresentou desempenho limitado, indicando que relações puramente lineares explicam apenas parte do fenômeno analisado. A validação cruzada confirma essa baixa capacidade preditiva de forma consistente. Em contrapartida, os coeficientes são estáveis e facilmente interpretáveis, reforçando seu papel analítico. Assim, funciona bem como baseline e instrumento explicativo, mas não como melhor modelo preditivo.

## 6. Comparação de Desempenho dos Modelos

In [ ]:
# Tabela comparativa
comparacao_modelos = pd.DataFrame({
    'Modelo': ['Random Forest', 'XGBoost', 'Ridge Regression'],
    'R² Teste': [r2_test_rf, r2_test_xgb, r2_test_ridge],
    'R² CV (média)': [cv_scores_rf.mean(), cv_scores_xgb.mean(), cv_scores_ridge.mean()],
    'R² CV (std)': [cv_scores_rf.std(), cv_scores_xgb.std(), cv_scores_ridge.std()],
    'RMSE': [rmse_test_rf, rmse_test_xgb, rmse_test_ridge],
    'MAE': [mae_test_rf, mae_test_xgb, mae_test_ridge]
})

print("\n", comparacao_modelos.to_string(index=False))

# Visualização da comparação
fig_compare = go.Figure()

metricas = ['R² Teste', 'R² CV (média)', 'RMSE', 'MAE']
for i, modelo in enumerate(comparacao_modelos['Modelo']):
    valores = [
        comparacao_modelos.loc[i, 'R² Teste'],
        comparacao_modelos.loc[i, 'R² CV (média)'],
        comparacao_modelos.loc[i, 'RMSE'] / 0.3,  # Normalizar escala
        comparacao_modelos.loc[i, 'MAE'] / 0.3
    ]
    fig_compare.add_trace(go.Bar(
        name=modelo,
        x=metricas,
        y=valores
    ))

fig_compare.update_layout(
    title='Comparação de Desempenho dos Modelos',
    xaxis_title='Métrica',
    yaxis_title='Valor Normalizado',
    barmode='group',
    height=450
)
fig_compare.show()


           Modelo  R² Teste  R² CV (média)  R² CV (std)     RMSE      MAE
   Random Forest  0.318702       0.312618     0.045833 0.013183 0.010045
         XGBoost  0.311774       0.290661     0.042502 0.013250 0.010493
Ridge Regression  0.179224       0.175148     0.049333 0.014469 0.013180


### Valores reais vs preditos para todos os modelos


In [ ]:
fig_predictions = go.Figure()

fig_predictions.add_trace(go.Scatter(
    x=y_test, y=y_test_pred_rf,
    mode='markers',
    name='Random Forest',
    marker=dict(size=8, opacity=0.7)
))
fig_predictions.add_trace(go.Scatter(
    x=y_test, y=y_test_pred_xgb,
    mode='markers',
    name='XGBoost',
    marker=dict(size=8, opacity=0.7)
))
fig_predictions.add_trace(go.Scatter(
    x=y_test, y=y_test_pred_ridge,
    mode='markers',
    name='Ridge',
    marker=dict(size=8, opacity=0.7)
))

# Linha perfeita
y_min, y_max = y_test.min(), y_test.max()
fig_predictions.add_trace(go.Scatter(
    x=[y_min, y_max],
    y=[y_min, y_max],
    mode='lines',
    name='Predição Perfeita',
    line=dict(dash='dash', color='black', width=2)
))

fig_predictions.update_layout(
    title='Comparação: Valores Reais vs Preditos',
    xaxis_title='Valores Reais (IDHM)',
    yaxis_title='Valores Preditos',
    height=500
)
fig_predictions.show()

## 7. Síntese Comparativa dos Modelos

**Decisão de Modelo Recomendado**

| Aspecto                        | Random Forest | XGBoost    | Ridge Regression |
| ------------------------------ | ------------- | ---------- | ---------------- |
| **R² (Teste)**                 | 0.32          | 0.31       | 0.18             |
| **R² (Validação Cruzada)**     | 0.31          | 0.29       | 0.18             |
| Generalização                  | Boa           | Boa        | Limitada         |
| Overfitting                    | Leve          | Controlado | Mínimo           |
| Interpretabilidade             | Média         | Média      | Alta             |
| Complexidade                   | Média         | Alta       | Baixa            |
| Adequação a políticas públicas | Boa           | Boa        | Excelente        |


### Conclusão sobre os modelos

> Random Forest apresentou o melhor equilíbrio entre desempenho preditivo e estabilidade. O XGBoost teve desempenho semelhante, porém com maior complexidade de ajuste. O Ridge Regression mostrou menor poder preditivo, mas se destaca pela simplicidade e clareza interpretativa, sendo útil como baseline e ferramenta explicativa para políticas públicas.



### Ranking de importância das Variáveis (média dos 3 modelos):

In [ ]:
importance = feature_importance_rf.copy()
importance.columns = ['Feature', 'RF_importance']

# Normalizar as importâncias para escala 0-1
rf_norm = feature_importance_rf.set_index('Feature')['Importance'].to_dict()
xgb_norm = feature_importance_xgb.set_index('Feature')['Importance'].to_dict()

# Criar ranking
ranking = []
for feat in variaveis_preditoras:
    rf_imp = rf_norm.get(feat, 0)
    xgb_imp = xgb_norm.get(feat, 0)
    ridge_coef = abs(coeficientes_ridge[coeficientes_ridge['Feature'] == feat]['Coeficiente'].values[0])
    ridge_coef_norm = ridge_coef / coeficientes_ridge['Coeficiente'].abs().max()
    
    media = (rf_imp + xgb_imp + ridge_coef_norm) / 3
    ranking.append({
        'Feature': feat,
        'RF': rf_imp,
        'XGB': xgb_imp,
        'Ridge': ridge_coef_norm,
        'Média': media
    })

ranking_df = pd.DataFrame(ranking).sort_values('Média', ascending=False)

for i, row in ranking_df.iterrows():
    pct = row['Média'] * 100
    print(f"   {i+1}. {row['Feature']:20s}: {pct:5.1f}%")

# Visualização
fig_consolidado = px.bar(
    ranking_df,
    x='Média',
    y='Feature',
    orientation='h',
    title='Importância das Variáveis (Random Forest, XGBoost e Ridge)',
    labels={'Média': 'Importância Relativa (0-1)', 'Feature': 'Variável Socioeconômica'},
    color='Média',
    color_continuous_scale='Turbo'
)
fig_consolidado.update_layout(showlegend=False, height=400)
fig_consolidado.show()

   4. analfabetismo       :  58.9%
   2. internet            :  30.9%
   3. homicidios          :  27.3%
   1. log_pib_pc          :  13.4%


## 8. Conclusão

> Os resultados obtidos reforçam que o desenvolvimento humano municipal é um fenômeno multidimensional, fortemente influenciado por fatores sociais estruturais e não apenas por indicadores econômicos isolados. Modelos mais flexíveis evidenciaram que variáveis como alfabetização, acesso à internet e segurança pública exercem papel central na explicação do IDHM, indicando que políticas públicas eficazes precisam atuar de forma integrada sobre essas dimensões.
>
> Nesse contexto, investimentos em educação básica, especialmente na redução do analfabetismo, mostram-se estratégicos para promover ganhos sustentáveis de bem-estar. A educação funciona como base para ampliar oportunidades de renda, melhorar a inserção no mercado de trabalho e potencializar os efeitos de outras políticas públicas, gerando impactos de longo prazo no desenvolvimento humano.
>
> A inclusão digital surge como outro eixo fundamental, ao ampliar o acesso à informação, a serviços públicos e a oportunidades econômicas. A expansão da conectividade, sobretudo em municípios mais vulneráveis, contribui para reduzir desigualdades regionais e fortalecer a capacidade produtiva local, potencializando os efeitos de políticas educacionais e econômicas.
>
> Por fim, os achados indicam que políticas públicas mais eficazes devem ser planejadas de forma integrada, combinando educação, inclusão digital, geração de renda e segurança pública. A adoção de estratégias baseadas em evidências, com testes em contextos locais antes da expansão em larga escala, tende a aumentar a eficiência do gasto público e a produzir avanços mais consistentes no desenvolvimento humano municipal.

<div align="center">
    <img src="image.png" width="40%">
</div>